In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import pyspark.sql.functions as F
from pyspark.sql.functions import when, udf, col, regexp_replace, regexp_extract
from pyspark.sql.types import DoubleType,IntegerType, StringType 

In [2]:
sc = SparkContext( 'local' )  
sqlCtx = SQLContext( sc )

In [3]:
df = sqlCtx.read.csv('data/airline-passengers.csv', header=True, inferSchema =True)
df.toPandas()

,Month,Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121
...,...,...
139,1960-08,606
140,1960-09,508
141,1960-10,461
142,1960-11,390


In [4]:
df.printSchema()

root
 |-- Month: string (nullable = true)
 |-- Passengers: integer (nullable = true)



In [5]:
df1 = df.withColumn('parsed',F.to_timestamp('Month','yyyy-MM'))
df1.show()

+-------+----------+-------------------+
|  Month|Passengers|             parsed|
+-------+----------+-------------------+
|1949-01|       112|1949-01-01 00:00:00|
|1949-02|       118|1949-02-01 00:00:00|
|1949-03|       132|1949-03-01 00:00:00|
|1949-04|       129|1949-04-01 00:00:00|
|1949-05|       121|1949-05-01 00:00:00|
|1949-06|       135|1949-06-01 00:00:00|
|1949-07|       148|1949-07-01 00:00:00|
|1949-08|       148|1949-08-01 00:00:00|
|1949-09|       136|1949-09-01 00:00:00|
|1949-10|       119|1949-10-01 00:00:00|
|1949-11|       104|1949-11-01 00:00:00|
|1949-12|       118|1949-12-01 00:00:00|
|1950-01|       115|1950-01-01 00:00:00|
|1950-02|       126|1950-02-01 00:00:00|
|1950-03|       141|1950-03-01 00:00:00|
|1950-04|       135|1950-04-01 01:00:00|
|1950-05|       125|1950-05-01 00:00:00|
|1950-06|       149|1950-06-01 00:00:00|
|1950-07|       170|1950-07-01 00:00:00|
|1950-08|       170|1950-08-01 00:00:00|
+-------+----------+-------------------+
only showing top

In [6]:
df1.printSchema()

root
 |-- Month: string (nullable = true)
 |-- Passengers: integer (nullable = true)
 |-- parsed: timestamp (nullable = true)



In [7]:
# df1.where(df1['parsed']>= '1950').show()
df1.where( " parsed >= '1950-03' " ).show(5)  # 열 타입이 문자열이면 안됨.

+-------+----------+-------------------+
|  Month|Passengers|             parsed|
+-------+----------+-------------------+
|1950-03|       141|1950-03-01 00:00:00|
|1950-04|       135|1950-04-01 01:00:00|
|1950-05|       125|1950-05-01 00:00:00|
|1950-06|       149|1950-06-01 00:00:00|
|1950-07|       170|1950-07-01 00:00:00|
+-------+----------+-------------------+
only showing top 5 rows



In [8]:
df1.withColumn('y', F.year('parsed')).show(5)

+-------+----------+-------------------+----+
|  Month|Passengers|             parsed|   y|
+-------+----------+-------------------+----+
|1949-01|       112|1949-01-01 00:00:00|1949|
|1949-02|       118|1949-02-01 00:00:00|1949|
|1949-03|       132|1949-03-01 00:00:00|1949|
|1949-04|       129|1949-04-01 00:00:00|1949|
|1949-05|       121|1949-05-01 00:00:00|1949|
+-------+----------+-------------------+----+
only showing top 5 rows



In [9]:
df1.withColumn('m', F.month('parsed')).show(5)

+-------+----------+-------------------+---+
|  Month|Passengers|             parsed|  m|
+-------+----------+-------------------+---+
|1949-01|       112|1949-01-01 00:00:00|  1|
|1949-02|       118|1949-02-01 00:00:00|  2|
|1949-03|       132|1949-03-01 00:00:00|  3|
|1949-04|       129|1949-04-01 00:00:00|  4|
|1949-05|       121|1949-05-01 00:00:00|  5|
+-------+----------+-------------------+---+
only showing top 5 rows



In [10]:
df1.withColumn('f', F.date_format('parsed','yyyy년MM월')).show()

+-------+----------+-------------------+----------+
|  Month|Passengers|             parsed|         f|
+-------+----------+-------------------+----------+
|1949-01|       112|1949-01-01 00:00:00|1949년01월|
|1949-02|       118|1949-02-01 00:00:00|1949년02월|
|1949-03|       132|1949-03-01 00:00:00|1949년03월|
|1949-04|       129|1949-04-01 00:00:00|1949년04월|
|1949-05|       121|1949-05-01 00:00:00|1949년05월|
|1949-06|       135|1949-06-01 00:00:00|1949년06월|
|1949-07|       148|1949-07-01 00:00:00|1949년07월|
|1949-08|       148|1949-08-01 00:00:00|1949년08월|
|1949-09|       136|1949-09-01 00:00:00|1949년09월|
|1949-10|       119|1949-10-01 00:00:00|1949년10월|
|1949-11|       104|1949-11-01 00:00:00|1949년11월|
|1949-12|       118|1949-12-01 00:00:00|1949년12월|
|1950-01|       115|1950-01-01 00:00:00|1950년01월|
|1950-02|       126|1950-02-01 00:00:00|1950년02월|
|1950-03|       141|1950-03-01 00:00:00|1950년03월|
|1950-04|       135|1950-04-01 01:00:00|1950년04월|
|1950-05|       125|1950-05-01 00:00:00|1950

In [11]:
df1.selectExpr('year(parsed) as year', 'Passengers').groupBy('year').mean().orderBy('year').show()

+----+---------+------------------+
|year|avg(year)|   avg(Passengers)|
+----+---------+------------------+
|1949|   1949.0|126.66666666666667|
|1950|   1950.0|139.66666666666666|
|1951|   1951.0|170.16666666666666|
|1952|   1952.0|             197.0|
|1953|   1953.0|             225.0|
|1954|   1954.0|238.91666666666666|
|1955|   1955.0|             284.0|
|1956|   1956.0|            328.25|
|1957|   1957.0| 368.4166666666667|
|1958|   1958.0|             381.0|
|1959|   1959.0| 428.3333333333333|
|1960|   1960.0| 476.1666666666667|
+----+---------+------------------+



In [12]:
df1.groupBy(F.year('parsed').alias('year')).mean().orderBy('year').show()

+----+------------------+
|year|   avg(Passengers)|
+----+------------------+
|1949|126.66666666666667|
|1950|139.66666666666666|
|1951|170.16666666666666|
|1952|             197.0|
|1953|             225.0|
|1954|238.91666666666666|
|1955|             284.0|
|1956|            328.25|
|1957| 368.4166666666667|
|1958|             381.0|
|1959| 428.3333333333333|
|1960| 476.1666666666667|
+----+------------------+



In [13]:
df1.show()

+-------+----------+-------------------+
|  Month|Passengers|             parsed|
+-------+----------+-------------------+
|1949-01|       112|1949-01-01 00:00:00|
|1949-02|       118|1949-02-01 00:00:00|
|1949-03|       132|1949-03-01 00:00:00|
|1949-04|       129|1949-04-01 00:00:00|
|1949-05|       121|1949-05-01 00:00:00|
|1949-06|       135|1949-06-01 00:00:00|
|1949-07|       148|1949-07-01 00:00:00|
|1949-08|       148|1949-08-01 00:00:00|
|1949-09|       136|1949-09-01 00:00:00|
|1949-10|       119|1949-10-01 00:00:00|
|1949-11|       104|1949-11-01 00:00:00|
|1949-12|       118|1949-12-01 00:00:00|
|1950-01|       115|1950-01-01 00:00:00|
|1950-02|       126|1950-02-01 00:00:00|
|1950-03|       141|1950-03-01 00:00:00|
|1950-04|       135|1950-04-01 01:00:00|
|1950-05|       125|1950-05-01 00:00:00|
|1950-06|       149|1950-06-01 00:00:00|
|1950-07|       170|1950-07-01 00:00:00|
|1950-08|       170|1950-08-01 00:00:00|
+-------+----------+-------------------+
only showing top

In [14]:
df1.groupBy(F.window('parsed','1 week')).mean().show()

+--------------------+---------------+
|              window|avg(Passengers)|
+--------------------+---------------+
|[1949-06-30 10:00...|          148.0|
|[1951-09-27 09:00...|          162.0|
|[1959-08-27 09:30...|          463.0|
|[1955-07-28 09:30...|          347.0|
|[1959-09-24 08:30...|          407.0|
|[1949-08-25 10:00...|          136.0|
|[1949-12-29 09:00...|          115.0|
|[1959-01-29 08:30...|          342.0|
|[1954-07-29 08:30...|          293.0|
|[1957-04-25 08:30...|          355.0|
|[1953-12-31 09:00...|          204.0|
|[1956-11-29 08:30...|          306.0|
|[1953-11-26 09:00...|          201.0|
|[1958-03-27 08:30...|          348.0|
|[1949-09-29 09:00...|          119.0|
|[1955-01-27 08:30...|          233.0|
|[1956-06-28 09:30...|          413.0|
|[1957-12-26 08:30...|          340.0|
|[1958-08-28 09:30...|          404.0|
|[1954-02-25 09:00...|          235.0|
+--------------------+---------------+
only showing top 20 rows



In [15]:
!pip install pandas_datareader
from pandas_datareader import data

In [16]:
samsungdf = data.get_data_yahoo('005930.KS','2017-01-01' )
samsungdf.reset_index(inplace=True)

In [17]:
sdf = sqlCtx.createDataFrame(samsungdf)
sdf.show()

+-------------------+-------+-------+-------+-------+----------+---------------+
|               Date|   High|    Low|   Open|  Close|    Volume|      Adj Close|
+-------------------+-------+-------+-------+-------+----------+---------------+
|2017-01-02 00:00:00|36240.0|35880.0|35980.0|36100.0| 4650600.0|31899.654296875|
|2017-01-03 00:00:00|36620.0|36020.0|36280.0|36480.0| 7357650.0|32235.439453125|
|2017-01-04 00:00:00|36520.0|36100.0|36500.0|36160.0| 7971750.0| 31952.66796875|
|2017-01-05 00:00:00|36060.0|35540.0|36060.0|35560.0|1.096745E7|31422.482421875|
|2017-01-06 00:00:00|36440.0|36040.0|36180.0|36200.0| 8880950.0|   31988.015625|
|2017-01-09 00:00:00|37500.0|36560.0|36600.0|37220.0| 1.31949E7| 32889.33984375|
|2017-01-10 00:00:00|37400.0|37080.0|37280.0|37240.0| 9099800.0|  32907.0078125|
|2017-01-11 00:00:00|38560.0|37420.0|37520.0|38280.0|1.201815E7|  33826.0078125|
|2017-01-12 00:00:00|38800.0|37980.0|38000.0|38800.0|1.166915E7| 34285.48828125|
|2017-01-13 00:00:00|38320.0

In [18]:
sdf.groupBy(F.window('Date', '1 week')).mean('Close').orderBy('window').show()

+--------------------+------------------+
|              window|        avg(Close)|
+--------------------+------------------+
|[2016-12-29 09:00...|           36075.0|
|[2017-01-05 09:00...|           37548.0|
|[2017-01-12 09:00...|           37100.0|
|[2017-01-19 09:00...|           38544.0|
|[2017-01-26 09:00...|39313.333333333336|
|[2017-02-02 09:00...|           38928.0|
|[2017-02-09 09:00...|           37928.0|
|[2017-02-16 09:00...|           38788.0|
|[2017-02-23 09:00...|           38610.0|
|[2017-03-02 09:00...|           40060.0|
|[2017-03-09 09:00...|           41076.0|
|[2017-03-16 09:00...|           42224.0|
|[2017-03-23 09:00...|           41588.0|
|[2017-03-30 09:00...|           41740.0|
|[2017-04-06 09:00...|           41892.0|
|[2017-04-13 09:00...|           41252.0|
|[2017-04-20 09:00...|           42268.0|
|[2017-04-27 09:00...|45013.333333333336|
|[2017-05-04 09:00...|           46040.0|
|[2017-05-11 09:00...|           46116.0|
+--------------------+------------

### 연습문제

In [19]:
df14 = sqlCtx.read.csv('data/2014년 졸음운전 교통사고.csv', header=True, inferSchema =True, encoding='euc-kr')
df15 = sqlCtx.read.csv('data/2015년 졸음운전 교통사고.csv', header=True, inferSchema =True, encoding='euc-kr',)
df16 = sqlCtx.read.csv('data/2016년 졸음운전 교통사고.csv', header=True, inferSchema =True, encoding='euc-kr',)

1. 3개의 데이터를 병합하시오.

In [20]:
df = df14.unionAll(df15).unionAll(df16)
df.show()

+----------+--------+--------+--------+
|      구분|사고(건)|사망(명)|부상(명)|
+----------+--------+--------+--------+
| 2014년1월|     169|       7|     347|
| 2014년2월|     145|       3|     268|
| 2014년3월|     206|      12|     358|
| 2014년4월|     182|       4|     360|
| 2014년5월|     218|      17|     474|
| 2014년6월|     233|      13|     457|
| 2014년7월|     229|      14|     431|
| 2014년8월|     222|       9|     447|
| 2014년9월|     225|      15|     430|
|2014년10월|     210|      14|     392|
|2014년11월|     209|      10|     386|
|2014년12월|     178|      12|     329|
| 2015년1월|     205|       8|     375|
| 2015년2월|     175|       8|     375|
| 2015년3월|     237|       4|     492|
| 2015년4월|     227|      10|     486|
| 2015년5월|     231|       9|     504|
| 2015년6월|     207|       8|     401|
| 2015년7월|     273|       7|     556|
| 2015년8월|     268|      19|     534|
+----------+--------+--------+--------+
only showing top 20 rows



In [21]:
df = df14.unionAll( df15 )
df = df.unionAll( df16 )
df.show(df.count())

+----------+--------+--------+--------+
|      구분|사고(건)|사망(명)|부상(명)|
+----------+--------+--------+--------+
| 2014년1월|     169|       7|     347|
| 2014년2월|     145|       3|     268|
| 2014년3월|     206|      12|     358|
| 2014년4월|     182|       4|     360|
| 2014년5월|     218|      17|     474|
| 2014년6월|     233|      13|     457|
| 2014년7월|     229|      14|     431|
| 2014년8월|     222|       9|     447|
| 2014년9월|     225|      15|     430|
|2014년10월|     210|      14|     392|
|2014년11월|     209|      10|     386|
|2014년12월|     178|      12|     329|
| 2015년1월|     205|       8|     375|
| 2015년2월|     175|       8|     375|
| 2015년3월|     237|       4|     492|
| 2015년4월|     227|      10|     486|
| 2015년5월|     231|       9|     504|
| 2015년6월|     207|       8|     401|
| 2015년7월|     273|       7|     556|
| 2015년8월|     268|      19|     534|
| 2015년9월|     252|      10|     501|
|2015년10월|     214|      14|     475|
|2015년11월|     199|       3|     398|
|2015년12월|     21

1. 3개의 데이터를 병합하시요.

In [22]:
data = df.withColumn('parsed', F.to_timestamp('구분','yyyy년MM월'))
data.show(data.count())

+----------+--------+--------+--------+-------------------+
|      구분|사고(건)|사망(명)|부상(명)|             parsed|
+----------+--------+--------+--------+-------------------+
| 2014년1월|     169|       7|     347|2014-01-01 00:00:00|
| 2014년2월|     145|       3|     268|2014-02-01 00:00:00|
| 2014년3월|     206|      12|     358|2014-03-01 00:00:00|
| 2014년4월|     182|       4|     360|2014-04-01 00:00:00|
| 2014년5월|     218|      17|     474|2014-05-01 00:00:00|
| 2014년6월|     233|      13|     457|2014-06-01 00:00:00|
| 2014년7월|     229|      14|     431|2014-07-01 00:00:00|
| 2014년8월|     222|       9|     447|2014-08-01 00:00:00|
| 2014년9월|     225|      15|     430|2014-09-01 00:00:00|
|2014년10월|     210|      14|     392|2014-10-01 00:00:00|
|2014년11월|     209|      10|     386|2014-11-01 00:00:00|
|2014년12월|     178|      12|     329|2014-12-01 00:00:00|
| 2015년1월|     205|       8|     375|2015-01-01 00:00:00|
| 2015년2월|     175|       8|     375|2015-02-01 00:00:00|
| 2015년3월|     237|

In [23]:
data.printSchema()

root
 |-- 구분: string (nullable = true)
 |-- 사고(건): integer (nullable = true)
 |-- 사망(명): integer (nullable = true)
 |-- 부상(명): integer (nullable = true)
 |-- parsed: timestamp (nullable = true)



2. 년도 및 월별 평균 사망자를 보여주시오.

In [24]:
data.groupBy( F.year('parsed').alias('year') ).mean('사망(명)').orderBy('year').show()
data.groupBy( F.month('parsed').alias('month') ).mean('사망(명)').orderBy('month').show()

+----+------------------+
|year|     avg(사망(명))|
+----+------------------+
|2014|10.833333333333334|
|2015|               9.0|
|2016| 8.166666666666666|
+----+------------------+

+-----+------------------+
|month|     avg(사망(명))|
+-----+------------------+
|    1| 6.666666666666667|
|    2| 5.666666666666667|
|    3| 7.666666666666667|
|    4|               7.0|
|    5|              13.0|
|    6|              11.0|
|    7|              10.0|
|    8|11.666666666666666|
|    9|12.666666666666666|
|   10|12.666666666666666|
|   11|               6.0|
|   12|               8.0|
+-----+------------------+



3. 2015년 1월 ~ 7월 데이터를 보여주시오.

In [25]:
data.where("parsed >= '2015-01' and parsed <= '2015-08'").show()

+---------+--------+--------+--------+-------------------+
|     구분|사고(건)|사망(명)|부상(명)|             parsed|
+---------+--------+--------+--------+-------------------+
|2015년1월|     205|       8|     375|2015-01-01 00:00:00|
|2015년2월|     175|       8|     375|2015-02-01 00:00:00|
|2015년3월|     237|       4|     492|2015-03-01 00:00:00|
|2015년4월|     227|      10|     486|2015-04-01 00:00:00|
|2015년5월|     231|       9|     504|2015-05-01 00:00:00|
|2015년6월|     207|       8|     401|2015-06-01 00:00:00|
|2015년7월|     273|       7|     556|2015-07-01 00:00:00|
+---------+--------+--------+--------+-------------------+



4.  2016년 사고대비 사망율을 구하시오.

In [26]:
data.withColumn('사고대비 사망율', data['사망(명)']/data['사고(건)']).\
                select('`구분`','`사망(명)`','`사고(건)`','`사고대비 사망율`').\
                where( "parsed>='2016'").show()

+----------+--------+--------+--------------------+
|      구분|사망(명)|사고(건)|     사고대비 사망율|
+----------+--------+--------+--------------------+
| 2016년1월|       5|     192|0.026041666666666668|
| 2016년2월|       6|     174|0.034482758620689655|
| 2016년3월|       7|     217| 0.03225806451612903|
| 2016년4월|       7|     216|0.032407407407407406|
| 2016년5월|      13|     239| 0.05439330543933055|
| 2016년6월|      12|     200|                0.06|
| 2016년7월|       9|     227|0.039647577092511016|
| 2016년8월|       7|     230|0.030434782608695653|
| 2016년9월|      13|     187| 0.06951871657754011|
|2016년10월|      10|     183|  0.0546448087431694|
|2016년11월|       5|     200|               0.025|
|2016년12월|       4|     168|0.023809523809523808|
+----------+--------+--------+--------------------+



5. 2015년 대비  사망이 가장 많이 증가한 2016년 도 월을 구하시오.

In [27]:
data.select( F.year('parsed') , '사망(명)').show()

+------------+--------+
|year(parsed)|사망(명)|
+------------+--------+
|        2014|       7|
|        2014|       3|
|        2014|      12|
|        2014|       4|
|        2014|      17|
|        2014|      13|
|        2014|      14|
|        2014|       9|
|        2014|      15|
|        2014|      14|
|        2014|      10|
|        2014|      12|
|        2015|       8|
|        2015|       8|
|        2015|       4|
|        2015|      10|
|        2015|       9|
|        2015|       8|
|        2015|       7|
|        2015|      19|
+------------+--------+
only showing top 20 rows

